In [1]:
from math import exp
import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [2]:

zeta, tau, gamma =0.15913294, -0.09671497,  7.53715001


In [3]:
def get_fractional_series(X0):
    """
    :param X0: 待计算分数阶的序列
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr

In [4]:
data = pd.read_excel("京津冀3省2014-2023使用版数据-2位小数.xlsx")
n = data.shape[0]
train_n = n-2
data


,电子及通信设备制造业,主营业务收入（亿元）,R&D活动人员折合全时当量（人年）,R&D经费内部支出（亿元）
0,2014,1.837233,2.948889,5.300143
1,2015,1.630165,3.398392,6.479943
2,2016,1.507327,3.299884,6.701289
3,2017,1.596237,3.125379,7.232092
4,2018,1.685147,2.950875,7.762178
5,2019,1.753019,2.940913,7.183381
6,2020,1.933047,3.291241,8.746418
7,2021,2.519186,3.359376,9.791547
8,2022,2.522894,4.411321,13.322350
9,2023,2.564832,4.867823,16.109319


In [5]:
# 第一列是年份，故先舍弃
X0 = data.iloc[:train_n, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

In [6]:
# 构建 B
TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
ONE = np.ones(train_n - 1)
B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

# 构建 Y
Y = np.diff(y1)

# 最小二乘法
a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
a, b2, b3, h1, h2

(-0.027478068936147293,
 -0.0005491838990090679,
 0.00010703629123855608,
 7.536340413758641e-06,
 0.0014531948165078844)

In [7]:
# 求 mu
mu1 = (1-gamma)/(1+0.5*a*(1-gamma))
mu2 = (1-0.5*a*(1-gamma))/(1+0.5*a*(1-gamma))
mu3 = (h1*(1-gamma)*(exp(tau)-1))/(1+0.5*a*(1-gamma))/tau
mu4 = (h2*(1-gamma))/(1+0.5*a*(1-gamma))
mu1,mu2,mu3,mu4

(-5.998408200246729,
 0.835175325966469,
 -4.308879849770226e-05,
 -0.008716855703896935)

In [8]:
# 这里重新获取下原数据，以实现所有数据的预测
# 第一列是年份，故先舍弃
X0 = data.iloc[:, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
# 求 hat y1(1为上标的那个haty^1)
hat_y1 = [y1[0]]
for m in range(2, n + 1):
    hat_ym = 0
    hat_ym += mu2 ** (m - 1) * y1[0]
    for v in range(1, m):
        hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
    for w in range(0, m - 1):
        hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
    hat_y1.append(hat_ym)
print(hat_y1)

[0.018755054100109546, 0.014942813794624117, 0.011993457917795502, 0.009029560967667582, 0.005832390906452118, 0.00355604441518868, 0.001960569638660111, 0.0003453931347949142, 0.0002671684049146165, 0.00019622905440575423]


In [9]:
hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))
print(hat_x1)
hat_x0 = []
for k in range(2, n + 1):
    hat_x0_m = 0
    for j in range(0, k):
        hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
    hat_x0.append(hat_x0_m)
print(hat_x0)

[1.83723328 1.90221882 1.96728575 2.05459146 2.19665877 2.36937121
 2.59531049 3.38485909 3.52047555 3.69065028]
[1.6098544895668991, 1.541660311597451, 1.5388375612058849, 1.606421395714248, 1.7049686781808602, 1.8537811906771493, 2.5571074763234636, 2.512931295623733, 2.5671720256270567]


In [10]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)

print(APE)
print(MAPE)

1    0.012459
2    0.022777
3    0.035959
4    0.046718
5    0.027410
6    0.041006
7    0.015053
8    0.003949
9    0.000912
Name: 主营业务收入（亿元）, dtype: float64
0.02291594845182986


In [236]:
pd.DataFrame(hat_x0).to_excel('FNGBM_hatx0_case2.xlsx')
pd.DataFrame(APE).to_excel('FNGBM_APE_case2.xlsx')

4      28465.4700
5      35984.4000
6      43206.3000
7      52799.1000
8      59688.8000
9      67584.2000
10     78309.9325
11     87304.6806
12     92969.5665
13     98634.4524
14    100098.0000
15    110085.9746
16    132988.2635
17    149087.7480
Name: 主营业务收入（亿元）, dtype: float64